In [1]:
%matplotlib inline

# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [5]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
from functools import reduce
import os
import pickle

import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import sys
sys.path.insert(0, "D:\\Documents\\THU\\food_recipe_gen")

from recipe_1m_analysis.utils import Vocabulary, FOLDER_PATH, DATA_FILES 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
SOS_token = 0
EOS_token = 1

In [7]:
DATA_FILES

['allingrs_count.pkl',
 'allwords_count.pkl',
 'recipe1m_test.pkl',
 'recipe1m_vocab_ingrs.pkl',
 'recipe1m_vocab_toks.pkl']

In [8]:
with open(os.path.join(FOLDER_PATH,DATA_FILES[3]),'rb') as f:
    vocab_ingrs=pickle.load(f)
    
with open(os.path.join(FOLDER_PATH,DATA_FILES[4]),'rb') as f:
    vocab_tokens=pickle.load(f)

In [9]:
with open(os.path.join(FOLDER_PATH,DATA_FILES[2]),'rb') as f:
    data=pickle.load(f)

pairs=[]
for recipe in data:
    pairs.append([recipe["ingredients"],recipe["tokenized"]])

print(len(pairs))

18745


In [10]:
length=0
for i,pair in enumerate(pairs):
    length_t = len([item for sublist in pair[1] for item in sublist])
    
    if length_t>length:
        length=length_t

length

539

In [11]:
MAX_LENGTH=300

In [12]:
# pairs[16826]

In [13]:
def filterPair(p):
    length=0
    for ingr in p[0]:
        if ingr not in vocab_ingrs.word2idx:
            return False
        
    for sent in p[1]:
        
        for word in sent:
            # TODO check how steps tokenized ? Put into vocab ???
            if word not in vocab_tokens.word2idx:
                return False
        length+=len(sent)
    
    return length < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

pairs = filterPairs(pairs)
print(len(pairs))

14724


In [14]:
# vocab_ingrs.word2idx['quahogs']

<div class="alert alert-info"><h4>Note</h4><p>There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>`__.</p></div>

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [15]:
def list2idx(vocab, sentence):
    return [vocab.word2idx[word] for word in sentence]


def tensorFromSentence(vocab, sentence,instructions=False):
    if instructions:
        indexes=[]
        for sent in sentence:
            indexes.extend(list2idx(vocab, sent))
    else:
        indexes = list2idx(vocab, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(vocab_ingrs, pair[0])
    target_tensor = tensorFromSentence(vocab_tokens, pair[1],instructions=True)
    return (input_tensor, target_tensor)

The Seq2Seq Model
=================
With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.



In [16]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input_, hidden):
        embedded = self.embedding(input_).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




**Simple Decoder**


In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).




In [17]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

**Attention Decoder**

If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

.. figure:: https://i.imgur.com/1152PYf.png
   :alt:

Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

.. figure:: /_static/img/seq-seq-images/attention-decoder-network.png
   :alt:





In [18]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf>`__.


Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [28]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH, attention=False):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    decoded_words=[]

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        if attention:
            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                loss += criterion(decoder_output, target_tensor[di])
                decoder_input = target_tensor[di]  # Teacher forcing
                topv, topi = decoder_output.topk(1)
                decoded_words.append(vocab_tokens.idx2word[topi.item()])
        else:
            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden)
                loss += criterion(decoder_output, target_tensor[di])
                decoder_input = target_tensor[di]  # Teacher forcing
                
                topv, topi = decoder_output.topk(1)
                decoded_words.append(vocab_tokens.idx2word[topi.item()])
            
    else:
        # Without teacher forcing: use its own predictions as the next input
        if attention:
            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()  # detach from history as input
                
                decoded_words.append(vocab_tokens.idx2word[topi.item()])
                loss += criterion(decoder_output, target_tensor[di])
                if decoder_input.item() == EOS_token:
                    break
        else:
            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()  # detach from history as input
                
                decoded_words.append(vocab_tokens.idx2word[topi.item()])
                loss += criterion(decoder_output, target_tensor[di])
                if decoder_input.item() == EOS_token:
                    break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length,decoded_words

In [29]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [30]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01,attention=True):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss,decoded_words = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion,attention=attention)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            print(decoded_words)

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [31]:
plt.switch_backend('agg')


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [38]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(vocab_ingrs, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden)#, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(vocab_tokens.idx2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [39]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [40]:
# hidden_size = 256
# encoder1 = EncoderRNN(len(vocab_ingrs.idx2word.keys()), hidden_size).to(device)
# attn_decoder1 = AttnDecoderRNN(hidden_size, len(vocab_tokens.idx2word.keys()), dropout_p=0.1).to(device)

# trainIters(encoder1, attn_decoder1, 5000, print_every=100)

In [35]:
hidden_size = 256
encoder2 = EncoderRNN(len(vocab_ingrs.idx2word.keys()), hidden_size).to(device)
attn_decoder2 = DecoderRNN(hidden_size, len(vocab_tokens.idx2word.keys())).to(device)

trainIters(encoder2, attn_decoder2, 5000, print_every=100,attention=False)

1m 18s (- 63m 56s) (100 2%) 29.3815
['preheat', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'bake', 'sugar', 'sugar', 'sugar', 'sugar', 'bake', 'sugar', 'sugar', 'bake', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'bake', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'bake', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'bake', 'sugar', 'sugar', 'sugar', 'sugar', 'bake', 'sugar', 'sugar', 'bake', 'sugar', 'sugar', 'sugar', 'sugar', 'sugar', 'bake', 'bake']
2m 33s (- 61m 31s) (200 4%) 47.0226
['salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', '<end>', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', '<end>', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt', 'salt

['to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'bacon', 'to', 'to', 'to']
19m 10s (- 44m 43s) (1500 30%) 71.0018
['melt', 'layer', 'bottom', 'layer', 'layer', 'layer', 'layer', 'layer', 'bottom', 'layer', 'layer', 'layer', 'layer', 'bottom', 'layer', 'layer', 'layer', 'layer', 'bottom', 'layer', 'bottom', 'layer', 'layer', 'layer', 'bottom', 'layer', 'bottom', 'layer', 'layer', 'layer', 'bottom', 'layer', 'a', 'layer', 'layer', 'layer', 'layer', 'layer', 'layer', 'bottom', 'layer', 'bottom', 'layer', 'bottom', 'layer', 'layer', 'layer', 'layer', 'bottom', 'layer', 'bottom', 'layer', 'layer', 'layer', 'layer', 'layer', 'layer

['melt', 'scoop', 'scoop', 'scoop', 'scoop', 'uncovered', 'scoop', 'scoop', ',', 'uncovered', 'scoop', 'scoop', 'as', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'as', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'uncovered', 'scoop', 'will', 'uncovered', 'scoop', 'will', 'uncovered', 'scoop', 'uncovered', 'scoop', 'uncovered', 'pepper', 'scoop', 'scoop', 'scoop', 'uncovered', 'scoop', 'scoop', 'scoop', 'will', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'pepper', 'scoop', 'uncovered', 'scoop', 'uncovered', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'pepper', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'uncovered', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'scoop', 'sc

['preheat', 'the', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'baking', 'a', 'a', 'a', 'a', 'a', 'baking', 'a', 'baking', 'a', 'a', 'a', 'a', 'a', 'baking', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'the', 'the', 'a', 'baking', 'tablespoon', 'tablespoon', 'tablespoon', 'dip', 'tablespoon', 'baking', 'tablespoon', 'baking', 'tablespoon', 'tablespoon', "'s", 'dip', 'tablespoon', 'tablespoon', 'baking', 'tablespoon', 'tablespoon', "'s", 'tablespoon', 'tablespoon', 'tablespoon', 'tablespoon', 'dip', 'tablespoon', 'tablespoon', 'a', 'tablespoon', 'tablespoon', 'tablespoon', 'dip', 'the', 'tablespoon', 'tablespoon', 'a', 'tablespoon', 'baking', 'the', 'tablespoon', 'baking', 'tablespoon', 'tablespoon', 'baking', 'tablespoon', 'baking', 'tablespoon', 'baking', 'gratin', 'tablespoon', 'tablespoon', 'dip', 'tablespoon', 'tablespoon', 'a', 'tablespoon', 'gratin', 'tablespoon', 'tablespoon', "'s", 'tablespoon', 'tablespoon', 'tablespoon', 'tablespoon', 'tablespoon', 'dip', 'tablespoon', 'a',

In [37]:
evaluateRandomly(encoder2, attn_decoder2)

> ['butter', 'unsweetened_applesauce', 'raw_sugar', 'bananas', 'eggs', 'pure_vanilla_extract', 'unbleached_all_-_purpose_flour', 'baking_soda', 'quick_-_cooking_oats', 'semi_-_sweet_chocolate_chips']
= [['preheat', 'oven', 'to', '350f', '.'], ['coat', '2', 'baking', 'sheets', 'with', 'cooking', 'spray', '.'], ['in', 'mixing', 'bowl', ',', 'combine', 'butter', 'and', 'applesauce', ',', 'beating', 'with', 'an', 'electric', 'mixer', 'until', 'creamy', '.'], ['add', 'sugar', ',', 'bananas', ',', 'eggs', 'and', 'vanilla', ',', 'beating', 'until', 'smooth', '.'], ['gradually', 'add', 'flour', 'and', 'baking', 'soda', '.'], ['stir', 'in', 'oats', 'and', 'chocolate', 'chips', '.'], ['drop', 'teaspoonfuls', 'onto', 'baking', 'sheets', '.'], ['bake', '7-10', 'minutes', 'or', 'until', 'golden', 'brown', '.'], ['cool', 'on', 'a', 'wire', 'rack', '.']]


TypeError: forward() takes 3 positional arguments but 4 were given

Visualizing Attention
---------------------

A useful property of the attention mechanism is its highly interpretable
outputs. Because it is used to weight specific encoder outputs of the
input sequence, we can imagine looking where the network is focused most
at each time step.

You could simply run ``plt.matshow(attentions)`` to see attention output
displayed as a matrix, with the columns being input steps and rows being
output steps:




In [ ]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "je suis trop froid .")
plt.matshow(attentions.numpy())

For a better viewing experience we will do the extra work of adding axes
and labels:




In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


# evaluateAndShowAttention("elle a cinq ans de moins que moi .")

# evaluateAndShowAttention("elle est trop petit .")

# evaluateAndShowAttention("je ne crains pas de mourir .")

# evaluateAndShowAttention("c est un jeune directeur plein de talent .")